In [6]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests

# EXTRACT

In [7]:
youtube_file = "US_Youtube.csv"
youtubecsv_df = pd.read_csv(youtube_file)
youtubecsv_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [8]:
# import JSON file
with open('category_youtube.json') as json_file:
    json_data = json.load(json_file)

pprint(json_data)

{'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
            'id': '1',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Film & Animation'}},
           {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
            'id': '2',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Autos & Vehicles'}},
           {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
            'id': '10',
            'kind': 'youtube#videoCategory',
            'snippet': {'assignable': True,
                        'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                        'title': 'Music'}},
 

In [9]:

# loop through JSON to get list of ids & categories
items = json_data["items"]
ids = []
titles = []
for item in items:
    ids.append(item["id"])
    titles.append(item["snippet"]["title"])
titles

['Film & Animation',
 'Autos & Vehicles',
 'Music',
 'Pets & Animals',
 'Sports',
 'Short Movies',
 'Travel & Events',
 'Gaming',
 'Videoblogging',
 'People & Blogs',
 'Comedy',
 'Entertainment',
 'News & Politics',
 'Howto & Style',
 'Education',
 'Science & Technology',
 'Nonprofits & Activism',
 'Movies',
 'Anime/Animation',
 'Action/Adventure',
 'Classics',
 'Comedy',
 'Documentary',
 'Drama',
 'Family',
 'Foreign',
 'Horror',
 'Sci-Fi/Fantasy',
 'Thriller',
 'Shorts',
 'Shows',
 'Trailers']

# Transform

In [10]:
category_list = pd.DataFrame(
    {'cat_id': ids,
     'category': titles
    })
category_list

,cat_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [11]:
#merge JSON and CSV
youtubecsv_df = youtubecsv_df.rename(columns = {"category_id": "cat_id"})
youtubecsv_df.dtypes
category_list.dtypes
category_list["cat_id"] = category_list['cat_id'].astype(str).astype(int)
category_list.dtypes

cat_id       int64
category    object
dtype: object

In [12]:
merged_df = pd.merge(youtubecsv_df, category_list, how = "left", on="cat_id")
merged_df.head()

,video_id,trending_date,title,channel_title,cat_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Entertainment


In [25]:
# Create a filtered dataframe from specific columns
youtube_cols = ["video_id", "cat_id", "views", "likes", "dislikes", "comment_count", "category"]
youtube_df = merged_df[youtube_cols].copy()

youtube_df.head()

,video_id,cat_id,views,likes,dislikes,comment_count,category
0,2kyS6SvSYSE,22,748374,57527,2966,15954,People & Blogs
1,1ZAPwfrtAFY,24,2418783,97185,6146,12703,Entertainment
2,5qpjK5DgCt4,23,3191434,146033,5339,8181,Comedy
3,puqaWrEC7tY,24,343168,10172,666,2146,Entertainment
4,d380meD0W0M,24,2095731,132235,1989,17518,Entertainment


# LOAD

In [ ]:
from sqlalchemy import create_engine

In [32]:
rds_connection_string = "postgres:Login@2210@localhost:5432/ETLPROJECT"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
engine.table_names()

['youtube_data', 'billboard_data']

In [34]:
youtube_df.to_sql(name='youtube_data', con=engine , if_exists='append' , index=False)

In [35]:
pd.read_sql_query('select * from youtube_data' , con=engine).head()

,video_id,cat_id,views,likes,dislikes,comment_count,category
0,2kyS6SvSYSE,22,748374,57527,2966,15954,People & Blogs
1,1ZAPwfrtAFY,24,2418783,97185,6146,12703,Entertainment
2,5qpjK5DgCt4,23,3191434,146033,5339,8181,Comedy
3,puqaWrEC7tY,24,343168,10172,666,2146,Entertainment
4,d380meD0W0M,24,2095731,132235,1989,17518,Entertainment
